# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev1 toc-item"><a href="#Generate-list-of-data" data-toc-modified-id="Generate-list-of-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate list of data</a></div><div class="lev2 toc-item"><a href="#Load-Data" data-toc-modified-id="Load-Data-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Data</a></div><div class="lev2 toc-item"><a href="#Organize-DataFrame" data-toc-modified-id="Organize-DataFrame-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Organize DataFrame</a></div><div class="lev1 toc-item"><a href="#Rank-Subgraphs-Based-on-Sparsity" data-toc-modified-id="Rank-Subgraphs-Based-on-Sparsity-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Rank Subgraphs Based on Sparsity</a></div><div class="lev1 toc-item"><a href="#Plot-Ranked-System-Subgraph-Matrices" data-toc-modified-id="Plot-Ranked-System-Subgraph-Matrices-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Plot Ranked System Subgraph Matrices</a></div><div class="lev1 toc-item"><a href="#Task-Related-Constrasts" data-toc-modified-id="Task-Related-Constrasts-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Task-Related Constrasts</a></div><div class="lev2 toc-item"><a href="#Check-Biases" data-toc-modified-id="Check-Biases-51"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Check Biases</a></div><div class="lev3 toc-item"><a href="#Task-Type" data-toc-modified-id="Task-Type-511"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>Task Type</a></div><div class="lev3 toc-item"><a href="#Interaction-Type" data-toc-modified-id="Interaction-Type-512"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>Interaction Type</a></div><div class="lev2 toc-item"><a href="#Between-Task-Subgraph-Contrast" data-toc-modified-id="Between-Task-Subgraph-Contrast-52"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Between-Task Subgraph Contrast</a></div><div class="lev2 toc-item"><a href="#Within-Task-Subgraph-Contrast" data-toc-modified-id="Within-Task-Subgraph-Contrast-53"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Within-Task Subgraph Contrast</a></div><div class="lev1 toc-item"><a href="#Task-Performance" data-toc-modified-id="Task-Performance-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Task Performance</a></div><div class="lev2 toc-item"><a href="#Behavioral-Correlation" data-toc-modified-id="Behavioral-Correlation-61"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Behavioral Correlation</a></div><div class="lev2 toc-item"><a href="#Regional-Contribution-to-Modulation-in-Performance" data-toc-modified-id="Regional-Contribution-to-Modulation-in-Performance-62"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Regional Contribution to Modulation in Performance</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'
import sys
import glob

import numpy as np
import pandas as pd
#import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib
import matplotlib.pyplot as plt

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(matplotlib.rcParams)

path_Remotes = '/Users/akhambhati/Remotes'
path_CoreData = path_Remotes + '/CORE.fMRI_cogcontrol.medaglia'
path_PeriphData = path_Remotes + '/RSRCH.NMF_CogControl'
path_InpData = path_PeriphData + '/e02b-FuncSubg'
path_ExpData = path_PeriphData + '/e03-FuncSubg_Dynamics'
path_Figures = './e03-Figures/'

for path in [path_CoreData, path_PeriphData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)

# Generate list of data

## Load Data

In [ ]:
%matplotlib inline

# Load Subgraph Data
df_system = np.load('{}/Subgraph.All.npz'.format(path_InpData))
sys_names = df_system['system_names']
sys_lbl = df_system['system_labels']
atlas_lbl = df_system['lausanne_labels']
sys_subgraph = df_system['system_subgraph']

n_subgraph = len(sys_subgraph)
key_type = df_system['task_key']

task_order = ['Stroop', 'Navon']
intr_order = ['pos', 'neg']
cond_order = ['rs', 'lo', 'hi']
n_subj = sys_subgraph[0]['expr_coef'].shape[0]
n_block = sys_subgraph[0]['expr_coef'].shape[2]


# Load Behavioral Data
df_blk = io.loadmat('{}/BlockwiseDataCorrectTrialsOnly.mat'.format(path_CoreData))
bad_subj_ix = [1, 6]
good_subj_ix = np.setdiff1d(np.arange(n_subj+2), bad_subj_ix)
df_perf = {'Stroop': {'lo': df_blk['StroopData'][good_subj_ix, 4, :],
                      'hi': df_blk['StroopData'][good_subj_ix, 2, :]},
           'Navon' : {'lo': df_blk['NavonData'][good_subj_ix, 4, :],
                      'hi': df_blk['NavonData'][good_subj_ix, 2, :]}}

"""
'high control accuracy', 'low control accuracy', 'high control mean RT',
'high control median RT', 'low control mean RT', 'low control median RT'
"""


# Load Motion Data
df_motion = {'Stroop': io.loadmat('{}/StroopMove.mat'.format(path_CoreData))['move'][:, 0],
             'Navon': io.loadmat('{}/NavonMove.mat'.format(path_CoreData))['move'][:, 0]}

## Organize DataFrame

In [ ]:
fac_expr_dict = {'Subgraph_ID': [],
                 'Subject_ID': [],
                 'Block_ID': [],
                 'Performance': [],
                 'Task_Type': [],
                 'Intr_Type': [],
                 'Cond_Type': [],
                 'Expression': []}

for fac_ii in xrange(n_subgraph):    
    for task in task_order:        
        for intr in intr_order:
            for cond in cond_order:
                key_ix = np.flatnonzero(key_type == 'adj_{}_{}_{}'.format(cond, intr, task))
                subj_coef = sys_subgraph[fac_ii]['expr_coef'][:, key_ix, :].squeeze()
                                
                for subj_id in xrange(n_subj):
                    for block_id in xrange(n_block):
                        
                        # performance
                        try:
                            perf = df_perf[task][cond][subj_id, block_id]
                        except:
                            perf = np.nan

                        fac_expr_dict['Subgraph_ID'].append(fac_ii)
                        fac_expr_dict['Subject_ID'].append(subj_id)
                        fac_expr_dict['Block_ID'].append(block_id)
                        fac_expr_dict['Performance'].append(perf)
                        fac_expr_dict['Task_Type'].append(task)
                        fac_expr_dict['Intr_Type'].append(intr)
                        fac_expr_dict['Cond_Type'].append(cond)
                        fac_expr_dict['Expression'].append(subj_coef[subj_id, block_id]) 
df = pd.DataFrame(fac_expr_dict, columns=fac_expr_dict.keys())

# Rank Subgraphs Based on Sparsity

In [ ]:
expression_sparsity = np.array([np.mean(sys_subgraph[fac_ii]['expr_coef'] == 0)
                                for fac_ii in xrange(n_subgraph)])

subgraph_rank = np.argsort(expression_sparsity)[::-1]
letter_lbl = np.array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'])
for fac_ii in xrange(n_subgraph):
    print(subgraph_rank[fac_ii], letter_lbl[fac_ii])
    
    
plt.figure()
ax = plt.subplot(111)
ax.plot(expression_sparsity[subgraph_rank])
ax.set_xlim([-0.5, 16.5])
ax.set_ylim([0, 1.0])

ax.set_xlabel('Ranked Subgraphs')
ax.set_ylabel('Expression Sparsity')
ax.set_xticks(np.arange(0, 16))
ax.set_xticklabels(letter_lbl)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')

plt.savefig('{}/Subgraph_Rank.svg'.format(path_Figures))

plt.show()    


# Plot Ranked System Subgraph Matrices

In [ ]:
%matplotlib inline

# Colormap thresholds
all_sys_edge = np.array([fac['sys_subg'].reshape(-1) for fac in sys_subgraph]).reshape(-1)
vmin = np.percentile(all_sys_edge[np.nonzero(all_sys_edge)], 10)
vmax = np.percentile(all_sys_edge[np.nonzero(all_sys_edge)], 90)

fsize = 5.5
fig = plt.figure(figsize=(6.9, 6.9), dpi=300)
for ii in xrange(n_subgraph):
    ax = fig.add_subplot(4, 4, ii+1)
    mat = ax.matshow(sys_subgraph[subgraph_rank[ii]]['sys_subg'], cmap='Reds') #, vmin=vmin, vmax=vmax)
    
    ax.set_yticks(xrange(len(sys_names)))
    ax.set_xticks(xrange(len(sys_names)))    
    if ii+1 in [1,5,9]:
        ax.set_xticklabels([])
        ax.set_yticklabels(sys_names, fontsize=fsize)      
    elif ii+1 in [14,15,16]:
        ax.set_yticklabels([])   
        ax.set_xticklabels(sys_names, fontsize=fsize, rotation=90)        
    elif ii+1 in [13]:  
        ax.set_yticklabels(sys_names, fontsize=fsize)      
        ax.set_xticklabels(sys_names, fontsize=fsize, rotation=90)        
    else:
        ax.set_xticklabels([])
        ax.set_yticklabels([]) 
    ax.set_title('Subgraph {}'.format(letter_lbl[ii]), fontsize=fsize)
        
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
fig.tight_layout(pad=0.01, h_pad=0.001, w_pad=0.001)
fig.savefig('{}/Ranked_System_Subgraph.svg'.format(path_Figures))
fig.show()

# Task-Related Constrasts

## Check Biases

### Task Type

In [ ]:
subj_task = df.groupby(['Subject_ID', 'Task_Type']).mean().unstack()
print(stats.ttest_rel(subj_task['Expression']['Navon'],
                      subj_task['Expression']['Stroop']))

### Interaction Type

In [ ]:
subj_task = df.groupby(['Subject_ID', 'Intr_Type']).mean().unstack()
print(stats.ttest_rel(subj_task['Expression']['pos'],
                      subj_task['Expression']['neg']))

## Between-Task Subgraph Contrast

In [ ]:
%matplotlib inline
    
# Subgraph/Task Bias Plot
for intr in df['Intr_Type'].unique():
    sel_intr = df[df['Intr_Type'] == intr]
    subg_task = sel_intr.groupby(['Subgraph_ID', 'Subject_ID', 'Cond_Type', 'Task_Type']).mean().unstack()
    
    # Get the expression difference
    diff_distrib_task = (subg_task['Expression']['Stroop'] - subg_task['Expression']['Navon']).unstack()
    diff_distrib = np.array((0.5*diff_distrib_task['hi'] + 0.5*diff_distrib_task['lo']).unstack())
    diff_means = diff_distrib.mean(axis=1)
    diff_stderrs = diff_distrib.std(axis=1) / np.sqrt(diff_distrib.shape[1]) 
    
    print(stats.f_oneway(*diff_distrib))
        
    # Permutation test
    diff_distrib_null = []
    for n_i in xrange(1000):
        diff_distrib_null.append(np.random.permutation(diff_distrib.reshape(-1)).reshape(n_subgraph, -1).mean(axis=1))
    min_null_thr = np.percentile(diff_distrib_null, 1.25)
    max_null_thr = np.percentile(diff_distrib_null, 98.75)    
    
    # Color based on permutation test
    colors = []
    for fac_ii in xrange(n_subgraph):
        if diff_means[fac_ii] < min_null_thr:
            colors.append('r')
        elif diff_means[fac_ii] > max_null_thr:
                colors.append('b')
        else:
            colors.append('k')
    colors = np.array(colors)    

    # Reorder factors based on the expression difference
    ord_ix = np.argsort(diff_means)[::-1]
    subgraph_rank_ord = np.array([np.flatnonzero(subgraph_rank == o_ii)[0]
                                  for o_ii in ord_ix])
    print(ord_ix)
    print(letter_lbl[subgraph_rank_ord])
    
    # Multiple T-Tests
    n_comp = n_subgraph * (n_subgraph-1) * 0.5
    alpha = 0.05 / n_comp
    t_table = np.zeros((n_subgraph, n_subgraph))
    for iix in xrange(n_subgraph):
        for iiy in xrange(n_subgraph):
            tval, pval = stats.ttest_rel(diff_distrib[iix, :],
                                         diff_distrib[iiy, :])
            if pval < alpha:
                t_table[ord_ix[iix], ord_ix[iiy]] = np.abs(tval)
        
    # Plot routine
    plt.figure(figsize=(3,3), dpi=300)
    ax = plt.subplot(111)
    ax.barh(np.arange(n_subgraph),
            diff_means[ord_ix],
            xerr=diff_stderrs[ord_ix], 
            color=colors[ord_ix])
    
    ax.fill_between(np.linspace(min_null_thr, max_null_thr, 100),
                    y1=-1, y2=n_subgraph, lw=0, alpha=0.2)
    ax.vlines(0, -1, n_subgraph, color='k')
    
    max_expr = np.max(np.abs(diff_means)) + np.max(diff_stderrs)
    ax.set_xlim([-1.1*max_expr, 1.1*max_expr])
    ax.set_ylim([-1, n_subgraph])
    
    ax.set_xlabel('Difference in Expression')
    ax.set_ylabel('Ranked Subgraphs')
    ax.set_title(intr)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    
    plt.savefig('{}/Task_Specificity.{}.svg'.format(path_Figures, intr))
    
    """
    plt.figure(figsize=(3,3), dpi=300)
    ax = plt.subplot(111)
    ax.matshow(t_table, cmap='Reds')
    ax.set_xticks(np.arange(n_subgraph))
    ax.set_xticklabels(letter_lbl[subgraph_rank_ord])
    ax.set_yticks(np.arange(n_subgraph))    
    ax.set_yticklabels(letter_lbl[subgraph_rank_ord])
    """
    
plt.show()    

## Within-Task Subgraph Contrast

In [ ]:
from matplotlib.patches import Ellipse
clr = [plt.cm.Set1(float(ii)/n_subgraph) for ii in xrange(n_subgraph)]


for task in df['Task_Type'].unique():
    
    sel_task = df[df['Task_Type'] == task]
    subg_task = sel_task.groupby(['Subgraph_ID', 'Subject_ID', 'Intr_Type', 'Cond_Type']).mean().unstack()

    rs_pos = np.array(subg_task['Expression']['rs'].unstack()['pos'].unstack())
    rs_neg = np.array(subg_task['Expression']['rs'].unstack()['neg'].unstack())    
    hi_pos = np.array(subg_task['Expression']['hi'].unstack()['pos'].unstack())
    hi_neg = np.array(subg_task['Expression']['hi'].unstack()['neg'].unstack())
    lo_pos = np.array(subg_task['Expression']['lo'].unstack()['pos'].unstack())
    lo_neg = np.array(subg_task['Expression']['lo'].unstack()['neg'].unstack())
            
    hi_perf = subg_task['Performance']['hi'].unstack()['pos'].unstack().mean(axis=0)
    lo_perf = subg_task['Performance']['lo'].unstack()['pos'].unstack().mean(axis=0)
    cost_perf = hi_perf-lo_perf    
    
    prf_pairs = [('Lo-Rs', lo_perf), ('Hi-Lo', cost_perf)]
    pos_pairs = [[lo_pos, rs_pos], [hi_pos, lo_pos]]
    neg_pairs = [[lo_neg, rs_neg], [hi_neg, lo_neg]]
    
    
    print('\n\n\n\n\n*****{}*****'.format(task))
    for prf, pos, neg in zip(prf_pairs, pos_pairs, neg_pairs):
        
        pos_idx = (pos[0]-pos[1])
        neg_idx = (neg[0]-neg[1])
        
        pos_idx_mean = pos_idx.mean(axis=1)
        neg_idx_mean = neg_idx.mean(axis=1)        
        pos_idx_stderr = pos_idx.std(axis=1) / np.sqrt(n_subj)
        neg_idx_stderr = neg_idx.std(axis=1) / np.sqrt(n_subj)        
        
        real_m, real_b, _, _, _ = stats.linregress(pos_idx_mean, neg_idx_mean)
        real_rho, real_pval = stats.spearmanr(pos_idx_mean, neg_idx_mean)

        """
        ### Permutation Tests
        null_rho_dist = []
        for n_i in xrange(10000):
            pos_null_1 = np.random.permutation(pos[0].reshape(-1)).reshape(n_subgraph, n_subj)
            pos_null_2 = np.random.permutation(pos[1].reshape(-1)).reshape(n_subgraph, n_subj)
            neg_null_1 = np.random.permutation(neg[0].reshape(-1)).reshape(n_subgraph, n_subj)
            neg_null_2 = np.random.permutation(neg[1].reshape(-1)).reshape(n_subgraph, n_subj)
            
            pos_null_idx = (pos_null_1-pos_null_2).mean(axis=1)
            neg_null_idx = (neg_null_1-neg_null_2).mean(axis=1)            
            
            null_rho, _ = stats.spearmanr(pos_null_idx, neg_null_idx)
            null_rho_dist.append(null_rho)

        null_rho_dist = np.array(null_rho_dist)
        if real_rho > 0:
            real_pval = np.mean(null_rho_dist > real_rho)
        else:
            real_pval = np.mean(null_rho_dist < real_rho)
        """
            
        ## Print out salient control subgraphs
        # Find the subgraphs in each quadrant
        q1 = np.intersect1d(np.flatnonzero(pos_idx_mean > 0),
                            np.flatnonzero(neg_idx_mean > 0))
        q2 = np.intersect1d(np.flatnonzero(pos_idx_mean < 0),
                            np.flatnonzero(neg_idx_mean > 0))
        q3 = np.intersect1d(np.flatnonzero(pos_idx_mean < 0),
                            np.flatnonzero(neg_idx_mean < 0))
        q4 = np.intersect1d(np.flatnonzero(pos_idx_mean > 0),
                            np.flatnonzero(neg_idx_mean < 0))

        # Order by distance away from (0,0)
        for qi, qq in enumerate([q1, q2, q3, q4]):
            qq_dist = qq[np.argsort(np.sqrt(pos_idx_mean[qq]**2 + neg_idx_mean[qq]**2))]  
            qq_letter = [letter_lbl[np.flatnonzero(subgraph_rank == subg_id)][0]
                         for subg_id in qq_dist]
            print('Quadrant {}: {}'.format(qi+1, qq_letter))
        
        
        # First, plot the 2D real distribution and ellipsoid errors
        plt.figure()
        ax = plt.subplot(111)
        
        for ii, fac_ii in enumerate(subgraph_rank):
            """
            ang = (np.arctan2(pos_mod_idx_mean[fac_ii] - pos_mod_idx[fac_ii, :],
                              neg_mod_idx_mean[fac_ii] - neg_mod_idx[fac_ii, :])
                   * 180/np.pi)


            ell = Ellipse(xy=(pos_mod_idx_mean[fac_ii],
                              neg_mod_idx_mean[fac_ii]),
                          width=pos_mod_idx_stderr[fac_ii],
                          height=neg_mod_idx_stderr[fac_ii],
                          angle=np.nanmean(ang))

            #ax.add_artist(ell)
            ell.set_alpha(0.4)
            ell.set_facecolor(clr[ii])
            ell.set_linewidth(0.0)
            """
            ax.scatter(pos_idx_mean[fac_ii], neg_idx_mean[fac_ii],
                       s=50, lw=0, color=clr[ii])        
        
        ax.legend(letter_lbl)  

        # Next, plot the linear fits
        x = np.array([-2.5, 2.5])
        y = real_m*x + real_b
        ax.plot(x, y, color='k', lw=1.0)   
        ax.text(1.0, 0.3, 'rho=%0.3f\np=%0.3f' % (real_rho, real_pval))     
        
        ax.set_xlim([-3.2, 3.2])
        ax.set_ylim([-0.6, 0.6])
        
        plt.savefig('{}/{}_Control_Matrix.{}.svg'.format(path_Figures, task, prf[0]))
        
        plt.show()

# Task Performance

## Behavioral Correlation

In [ ]:
subgraph_beh_path = '{}/Subgraph.Behavior_Corr.npz'.format(path_ExpData)
if not os.path.exists(subgraph_beh_path):
    beh_stats_table = np.load(subgraph_beh_path)['stats_table'][()]
else:
    import statsmodels.api as sm
    import statsmodels.graphics.regressionplots as regplot

    beh_stats_table = {'Subgraph_ID': [],
                       'Task': [],
                       'Task_Cond': [],
                       'Expr_Type': [],
                       'rho': [],
                       'pval': []}
    
    for task in df['Task_Type'].unique():
        sel_task = df[df['Task_Type'] == task]
        subg_task = sel_task.groupby(['Subgraph_ID', 'Subject_ID', 'Intr_Type', 'Cond_Type']).mean().unstack()

        rs_pos = np.array(subg_task['Expression']['rs'].unstack()['pos'].unstack())
        rs_neg = np.array(subg_task['Expression']['rs'].unstack()['neg'].unstack())    
        hi_pos = np.array(subg_task['Expression']['hi'].unstack()['pos'].unstack())
        hi_neg = np.array(subg_task['Expression']['hi'].unstack()['neg'].unstack())
        lo_pos = np.array(subg_task['Expression']['lo'].unstack()['pos'].unstack())
        lo_neg = np.array(subg_task['Expression']['lo'].unstack()['neg'].unstack())

        hi_perf = subg_task['Performance']['hi'].unstack()['pos'].unstack().mean(axis=0)
        lo_perf = subg_task['Performance']['lo'].unstack()['pos'].unstack().mean(axis=0)
        cost_perf = hi_perf-lo_perf    

        prf_pairs = [('Lo-Rs', lo_perf), ('Hi-Lo', cost_perf)]
        pos_pairs = [[lo_pos, rs_pos], [hi_pos, lo_pos]]
        neg_pairs = [[lo_neg, rs_neg], [hi_neg, lo_neg]]

        print('\n\n\n\n\n*****{}*****'.format(task))
        for fac_ii in xrange(n_subgraph):
            llbl = letter_lbl[subgraph_rank == fac_ii][0]
            print(llbl)

            for prf, pos, neg in zip(prf_pairs, pos_pairs, neg_pairs):
                model1 = sm.OLS(prf[1], df_motion[task].T)
                res1 = model1.fit().resid           

                for mod_dir in [('Pos', pos),
                                ('Neg', neg)]:
                    mod_idx = mod_dir[1][0][fac_ii, :]-mod_dir[1][1][fac_ii, :]

                    model2 = sm.OLS(mod_idx, df_motion[task].T)
                    res2 = mod_idx #model2.fit().resid           

                    real_rho, real_pval = stats.spearmanr(res2, res1)

                    real_m, real_b, _, _, _ = stats.linregress(res2, res1)
                    """
                    ### Permutation Tests
                    null_rho_dist = []
                    for n_i in xrange(10000):
                        null_expr = np.random.permutation(np.array(subg_task['Expression']).reshape(-1)).reshape(n_subgraph, n_subj, 6)
                        null_expr_1 = null_expr[fac_ii, :, 0]
                        null_expr_2 = null_expr[fac_ii, :, 1]
                        mod_idx_null = null_expr_1-null_expr_2

                        model3 = sm.OLS(mod_idx_null, df_motion[task].T)
                        res3 = mod_idx_null

                        null_rho, _ = stats.spearmanr(res3, res1)
                        null_rho_dist.append(null_rho)

                    null_rho_dist = np.array(null_rho_dist)
                    if real_rho > 0:
                        real_pval = np.mean(null_rho_dist > real_rho)
                    else:
                        real_pval = np.mean(null_rho_dist < real_rho)
                    """
                    
                    if real_pval < 0.05:
                        sig = '*'
                    else:
                        sig = ' '
                                        
                    # Cache and Print output
                    beh_stats_table['Subgraph_ID'].append(fac_ii)
                    beh_stats_table['Task'].append(task)
                    beh_stats_table['Task_Cond'].append(prf[0])
                    beh_stats_table['Expr_Type'].append(mod_dir[0])
                    beh_stats_table['rho'].append(real_rho)
                    beh_stats_table['pval'].append(real_pval)
                    
                    print('%s, %s, %0.4f, %0.4f, %s' % 
                          (prf[0], mod_dir[0], real_rho, real_pval, sig))

                    """
                    # First, plot the 2D real distribution and ellipsoid errors
                    plt.figure()
                    ax = plt.subplot(111)

                    ax.scatter(res2, res1,
                               s=20, lw=0, color=[0.5, 0.5, 0.5])        


                    # Next, plot the linear fits
                    x = np.array([res2.min(), res2.max()])
                    y = real_m*x + real_b
                    ax.plot(x, y, color='k', lw=1.0)   
                    ax.text(0.75*(x.max()+x.min()),
                            0.75*(y.max()+y.min()),
                            'r=%0.3f\np=%0.3f' % (real_rho, real_pval))  

                    # Get the axis boundaries
                    x_bound = 1.05*np.max([np.abs(res2.min()),
                                           np.abs(res2.max())])
                    y_bound = 1.05*np.max([np.abs(res1.min()),
                                           np.abs(res1.max())])
                    ax.set_xlim([-x_bound, x_bound])
                    ax.set_ylim([-y_bound, y_bound])                

                    plt.savefig('{}/{}.Subgraph_{}.{}.{}.svg'.format(path_Figures, task, llbl,
                                                                     prf[0], mod_dir[0]))

                    plt.close()
                    """
    df_stats_table = pd.DataFrame(beh_stats_table, columns=beh_stats_table.keys())
    

## Regional Contribution to Modulation in Performance

In [ ]:
all_subgraph = np.array([subg['full_subg'] for subg in sys_subgraph])
all_subg_cfg = convert_adj_matr_to_cfg_matr(all_subgraph)

roi_subg_partc = np.mean(all_subgraph, axis=2)

### Surrogate ROI Participation Coefficient
"""
n_perm = 1000
null_roi_subg_partc = []
for null_ii in xrange(n_perm):
    null_subg_cfg = np.array([convert_conn_vec_to_adj_matr(subg)
                              for subg in np.random.permutation(all_subg_cfg.T).T])    
    null_roi_subg_partc.append(np.mean(null_subg_cfg, axis=2))
null_roi_subg_partc = np.array(null_roi_subg_partc)
"""

### Project Subgraph Performance to the ROI Participation Coefficient
roi_corr_partc_thrshd = {}
for task in np.unique(df_stats_table['Task']):
    for task_cond in np.unique(df_stats_table['Task_Cond']):
        sel_stats = df_stats_table.loc[(df_stats_table['Task'] == task) & 
                                       (df_stats_table['Task_Cond'] == task_cond)]

        pos_rho = sel_stats[sel_stats['Expr_Type'] == 'Pos']['rho']
        neg_rho = -1*sel_stats[sel_stats['Expr_Type'] == 'Neg']['rho']
        subg_corr = (np.array(pos_rho) + np.array(neg_rho)) / 2.0
        behpos_ix = subg_corr > 0
        behneg_ix = subg_corr < 0        
        
        for perf_type in [('BehPos', behpos_ix),
                          ('BehNeg', behneg_ix)]:
            key_name = '{}.{}.{}'.format(task, task_cond, perf_type[0])
            
            if perf_type[0] == 'BehNeg':
                sel_subg_corr = -1*subg_corr[perf_type[1]]
            else:
                sel_subg_corr = subg_corr[perf_type[1]]
                
            roi_corr_partc = np.dot(sel_subg_corr, roi_subg_partc[perf_type[1], :])

            ### Threshold using the null distribution
            null_roi_corr_partc = []
            for null_ii in xrange(n_perm):
                null_roi_corr_partc.append(np.dot(sel_subg_corr,  null_roi_subg_partc[null_ii, :, :][perf_type[1], :]))
            null_roi_corr_partc = np.array([null_roi_corr_partc]).reshape(-1)

            null_max_thr = np.percentile(null_roi_corr_partc, 100)

            roi_corr_partc_thrshd[key_name] = roi_corr_partc.copy()
            roi_corr_partc_thrshd[key_name][roi_corr_partc < null_max_thr] = np.nan
            

            # Compute the histogram bin range using the
            # real and null distribution range
            xmin = roi_corr_partc.min()
            xmax = roi_corr_partc.max()
            real_bin_range = np.linspace(xmin, xmax, 50)            
            null_xmin = null_roi_corr_partc.min()
            null_xmax = null_roi_corr_partc.max()
            null_bin_range = np.linspace(null_xmin, null_xmax, 1000)            

            ### Histogram Plots            
            fig = plt.figure(figsize=(3,3), dpi=300)
            ax = fig.add_subplot(111)
            ax.hist(roi_corr_partc, bins=real_bin_range,
                    color='r', alpha=0.3, lw=0, normed=True)
            ax.hist(null_roi_corr_partc, bins=null_bin_range,
                    color='k', alpha=0.3, lw=0, normed=True)
            ax.vlines(null_xmin, 0, 800)
            ax.vlines(null_xmax, 0, 800)            
            ax.xaxis.set_ticks_position('bottom')
            ax.yaxis.set_ticks_position('left')
            ax.set_xlim([0.0, 0.01])
            ax.set_ylim([0.0, 5000])            
            ax.set_title(key_name)
            fig.savefig('{}/ROI_Partc_Perf.{}.svg'.format(path_Figures, key_name))
            plt.close()
            
            ### Print an ordered list of the significant brain regions
            print
            print
            print('***** {} *****'.format(key_name))
            good_ix = ~np.isnan(np.sort(roi_corr_partc_thrshd[key_name])[::-1])
            sig_roi = atlas_lbl[np.argsort(roi_corr_partc_thrshd[key_name])[::-1]][good_ix] 
            sig_roi, ind = np.unique([roi.split('_')[1] for roi in sig_roi], return_index=True)
            print(sig_roi[np.argsort(ind)])

In [ ]:
from matplotlib.offsetbox import AnnotationBbox, OffsetImage, TextArea
from mayavi import mlab
import nibabel as nib


view_angle = {'Sag_PA': [0.0, 90.0],
              'Sag_AP': [180.0, 90.0]}

# Get the pial surface recons
pial_hemi = {'LH': {},
             'RH': {}}
pial_hemi['LH']['vert'], pial_hemi['LH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/lh.pial'.format(path_CoreData))
pial_hemi['RH']['vert'], pial_hemi['RH']['tria'] = nib.freesurfer.io.read_geometry('{}/BrainRenderSubject15/surf/rh.pial'.format(path_CoreData))

# Get the Lausanne label files for each ROI
label_files = []
for roi in df_system['lausanne_labels']:
    hemi = roi.split('_')[0].lower()

    # Parse the atlas name and find the label file if it exists
    if len(roi.split('_')) == 2:
        lbl_file = ('%s.%s.label' % tuple(roi.split('_'))).lower()
    elif len(roi.split('_')) == 3:
        lbl_file = ('%s.%s_%s.label' % tuple(roi.split('_'))).lower()
    else:
        raise Exception
    lbl_file = lbl_file.replace(' ', '')

    label_files.append('{}/BrainRenderSubject15/label/regenerated_{}_125/{}'.format(path_CoreData, hemi, lbl_file))

    
# Iterate over experimental conditions
for corr_key in roi_corr_partc_thrshd.keys():
    xmin = np.percentile(roi_corr_partc_thrshd[corr_key][~np.isnan(roi_corr_partc_thrshd[corr_key])], 0)
    xmax = np.percentile(roi_corr_partc_thrshd[corr_key][~np.isnan(roi_corr_partc_thrshd[corr_key])], 95)
    
    pixmap = {}
    for hemi in pial_hemi.keys():
        n_vert = len(pial_hemi[hemi]['vert'])
        pial_scalars = np.zeros(n_vert)

        # Iterate over ROIs
        for roi_ix, (roi, lbl_file) in enumerate(zip(df_system['lausanne_labels'],
                                                     label_files)):
            if roi.split('_')[0] != hemi:
                continue

            if not os.path.exists(lbl_file):
                continue

            # Load the file and add scalar to the vertices
            parc_lbl = nib.freesurfer.io.read_label(lbl_file)   
            if not np.isnan(roi_corr_partc_thrshd[corr_key][roi_ix]):
                pial_scalars[parc_lbl] = roi_corr_partc_thrshd[corr_key][roi_ix]

        # Plot the colored Brain System
        fig = mlab.figure(bgcolor=(1.0, 1.0, 1.0))
        src = mlab.pipeline.triangular_mesh_source(pial_hemi[hemi]['vert'][:,0],
                                                   pial_hemi[hemi]['vert'][:,1],
                                                   pial_hemi[hemi]['vert'][:,2],
                                                   pial_hemi[hemi]['tria'], scalars=pial_scalars, opacity=0.45, figure=fig)
        norms = mlab.pipeline.poly_data_normals(src, figure=fig)
        norms.filter.splitting = False
        surf = mlab.pipeline.surface(norms, figure=fig)
        if corr_key.split('.')[-1] == 'BehNeg':
            cmap = 'Blues'
        else:
            cmap = 'Reds'
        surf.parent.scalar_lut_manager.set(lut_mode=cmap, data_range=[xmin, xmax], use_default_range=False)
        lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
        surf.module_manager.scalar_lut_manager.lut.table = lut

        # Rotate the view and save a screenshot
        for ang in view_angle.keys():
            mlab.view(azimuth=view_angle[ang][0],
                      elevation=view_angle[ang][1])
            pixmap['{}_{}'.format(hemi, ang)] = mlab.screenshot(mode='rgba')
        mlab.close(all=True)
    
    ### Plot the pixmap
    fig = plt.figure(figsize=(2,2), dpi=300)
    ax = fig.add_subplot(111)

    pixmap_key = [('LH_Sag_AP', False, (0.25, 0.575)),
                  ('RH_Sag_PA', False, (0.75, 0.575)),
                  ('LH_Sag_PA', True, (0.25, 0.225)),
                  ('RH_Sag_AP', True, (0.75, 0.225))]

    for px_key in pixmap_key:
        if px_key[1] == True:
            imagebox = OffsetImage(pixmap[px_key[0]][:, ::-1, :], zoom=0.65)
        else:
            imagebox = OffsetImage(pixmap[px_key[0]], zoom=0.65)
        imagebox.axes = fig.axes[0]
        ab = AnnotationBbox(imagebox, 
                            xy=px_key[2],
                            xycoords='data',
                            frameon=False,
                            pad=0.0)
        fig.axes[0].add_artist(ab)
    ax.set_axis_off()
    ax.set_title('{}\n{}'.format(corr_key, xmax))
    fig.savefig('{}/ROI_Partc_Perf.Atlas.{}.svg'.format(path_Figures, corr_key))
    fig.show()

In [ ]:
plt.figure()
ax = plt.subplot(211)
mat = ax.imshow(pixmap[px_key[0]], cmap='Blues')
ax.set_axis_off()
plt.colorbar(mat, ax=ax)

ax = plt.subplot(221)
mat = ax.imshow(pixmap[px_key[0]], cmap='Reds')
ax.set_axis_off()
plt.colorbar(mat, ax=ax)

plt.savefig('{}/Blues_Reds_CB.svg'.format(path_Figures))
plt.show()